In [435]:
import pandas as pd

# task_1
ratings_df = pd.read_csv("ratings.csv")
movies_df = pd.read_csv("movies.csv")

In [426]:
def rating_class(rating):
    if rating <= 2:
        return "low"
    elif rating <= 4:
        return "medium"
    elif 4.5 <= rating <= 5:
        return "high"
    else:
        return "no rating"

In [427]:
ratings_df["class"] = ratings_df["rating"].apply(rating_class)
ratings_df.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,medium
1,1,3,4.0,964981247,medium
2,1,6,4.0,964982224,medium
3,1,47,5.0,964983815,high
4,1,50,5.0,964982931,high


In [428]:
# task_2
keywords_df = pd.read_csv("keywords.csv")

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [429]:
import pymorphy3

geo_data = {
    "Центр": ["москва", "тула", "ярославль"],
    "Северо-Запад": ["петербург", "псков", "мурманск"],
    "Дальний Восток": ["владивосток", "сахалин", "хабаровск"],
}

word_morph = pymorphy3.MorphAnalyzer()


def region_classificator(kw: str):
    kw_normalized = [
        word_morph.parse(word)[0].normal_form for word in kw.lower().split(" ")
    ]

    regions = []
    for region, cities in geo_data.items():
        for city in cities:
            if city in kw_normalized:
                regions.append(region)

    return ", ".join(regions) if regions else "undefined"


keywords_df["region"] = keywords_df["keyword"].apply(region_classificator)

In [476]:
# task_3
import re

years = [year for year in range(1950, 2011)]


def production_year(movie_title):
    title_year = re.search(r"\((\d{4})\)", movie_title)

    if title_year:
        extracted_year = int(title_year.group(1))
        if extracted_year in years:
            return extracted_year if extracted_year else 1990


movies_df["year"] = movies_df["title"].apply(production_year)

movies_ratings_df = pd.merge(movies_df, ratings_df, on="movieId", how="inner")
years_mean_rating_df = (
    movies_ratings_df.groupby("year")["rating"]
    .mean()
    .reset_index()
    .sort_values(by="rating", ascending=True)
)
years_mean_rating_df.head()

,year,rating
46,1996.0,3.338894
47,1997.0,3.349749
42,1992.0,3.368166
55,2005.0,3.371513
50,2000.0,3.403993
